# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

In [1]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self._remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text


    def _remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = [] 
        for statement in context:
            if statement["role"] == "_user":
                new_statement = {"role": "USER", "message": statement["_content"]}
            elif statement["role"] == "_llm":
                new_statement = {"role": "CHATBOT", "message": statement["_content"]}
            elif statement["role"] == "_preamble":
                new_statement = {"role": "SYSTEM", "message": statement["_content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

/Users/justin.cechmanek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
from redisvl.extensions.session_manager import SessionManager
user_session = SessionManager(name='llm_chef', session_id='123', user_id='abc', application_id='chefbot_123')
user_session.set_preamble("You are a helpful chef, assisting people in making delicious meals")

client = CohereClient()

On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [8]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.fetch_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Sure! Here are a few breakfast ideas:

- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.
- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.
- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.
- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve them with some toast and fruit for a balanced meal.
- Pancakes or waffles: Make a batch of your favorite pancake or waffle batter and cook them up for a specia

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [9]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.fetch_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Sure, here's a basic recipe for pancakes that you can adjust and customize to your liking:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk
1 egg
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk, egg, and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve the pancakes warm with your favorite toppings such as butter, syrup, fruit, or whipped cream.

Feel free to

In [10]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

In [11]:
prompt = "I am vegetarian. Can you remove the eggs?"
context = user_session.fetch_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! Here's an egg-free version of the pancake recipe:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk (dairy or non-dairy alternative)
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve the pancakes warm with your favorite toppings such as butter, syrup, fruit, or whipped cream.

This version of the recip

In [12]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

In [14]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.fetch_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Sure, to make the pancake recipe vegan, you can replace the butter with a vegan alternative. Here's the updated recipe:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups plant-based milk (such as soy, almond, or oat milk)
3 tablespoons neutral-flavored oil (such as canola or vegetable oil), plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the plant-based milk and oil.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little oil.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve t

In [15]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that can break quickly.

In [17]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.fetch_recent(top_k=3)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure, here's the first recipe for pancakes that I provided: 

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk
1 egg
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk, egg, and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter; a few lumps are okay.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and small bubbles start to appear on the surface, about 2-3 minutes.
Flip the pancakes carefully and cook for an additional 1-2 minutes on the other side until golden brown.
Serve the pancakes warm with

In [18]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me the recipe for those pancakes?'}
{'role': '_llm', '_content': "Sure, here's a basic recipe for pancakes that you can adjust and customize to your liking:\n\nIngredients:\n\n1 1/2 cups all-purpose flour\n3 1/2 teaspoons baking powder\n1 teaspoon salt\n1 tablespoon sugar\n1 1/4 cups milk\n1 egg\n3 tablespoons melted butter, plus more for the pan\n\nInstructions:\n\nWhisk together the flour, baking powder, salt, and sugar in a large bowl.\nIn a separate bowl, whisk together the milk, egg, and melted butter.\nPour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.\nHeat a nonstick pan or griddle over medium heat and grease it with a little butter.\nPour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then f

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_context()` method.

In [52]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.fetch_context(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Sure, here's a simple recipe for avocado toast:

Ingredients:
- 1 ripe avocado
- 1 slice of your favorite bread
- Lemon juice, to taste
- Salt and pepper, to taste
- Optional: sliced tomato, a fried egg, or any other desired toppings

Directions:
1. Toast your bread to your desired level.
2. While the bread is toasting, smash the avocado in a bowl with a fork until it reaches your desired consistency. You can leave it chunkier or make it smoother, depending on your preference.
3. Add a squeeze of lemon juice to the avocado and season with salt and pepper to taste. Mix it all together.
4. Once the bread is toasted, spread the avocado mixture on top.
5. If desired, add any additional toppings such as sliced tomato or a fried egg.
6. Enjoy your delicious and nutritious avocado toast!

Feel free to experiment with different types of bread and additional toppings to create your own unique version of avocado toast!


In [53]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.